In [1]:
import gamspy as gp
import sys
import os
import pandas as pd
from gamspy import Container
from gamspy import Equation
from gamspy import Model
from gamspy import Parameter
from gamspy import Sense
from gamspy import Set
from gamspy import Sum
from gamspy import Variable



# Solving the Original Problem without Benders

In [2]:
def main():
    m  = Container(
        system_directory=os.getenv("SYSTEM_DIRECTORY", None),
        delayed_execution=int(os.getenv("DELAYED_EXECUTION", False)),
    )
    # Variables
    x = Variable(
        m,
        name="x",
        type="Positive"
    )
    y = Variable(
        m,
        name="y",
        type="Positive"
    )

    con1 = Equation(
        m,
        name="con1"
    )
    con2 = Equation(
        m,
        name="con2"
    )
    con3 = Equation(
        m,
        name="con3"
    )
    con4 = Equation(
        m,
        name="con4"
    )

    con1[...] = y - x <= 5
    con2[...] = y - (1/2)*x <= 15/2
    con3[...] = y + (1/2)*x <= (35/2)
    con4[...] = -y + x <= 10
    x.hi = 16
    original = Model(
        m,
        name="original",
        equations=m.getEquations(),
        problem="LP",
        sense=Sense.MIN,
        objective = -y - x/4
    )
    original.solve()
    print(x.records.level[0])
    print(y.records)
    print(original.objective_value)
    print(original.status)
    

if __name__ == "__main__":
    main()

10.0
   level  marginal  lower  upper  scale
0   12.5       0.0    0.0    inf    1.0
-15.0
ModelStatus.OptimalGlobal


# Solving the Same Problem with Benders

In [3]:
def main():
    m = Container(
        #working_directory=".",
        system_directory=os.getenv("SYSTEM_DIRECTORY", None),
        delayed_execution=int(os.getenv("DELAYED_EXECUTION", True)),
    )

    # Set
    iter = Set(
        m,
        name="iter",
        records = [f"{idx}" for idx in range(1,26)],
        description = "max Benders iterations",
    )
    activeIter = Set(
        m,
        name = "activeIter",
        domain = iter,
        description = "active Bender cuts",
        #records = [""]
    )
    
    # Variable
    x = Variable(
        m, 
        name = "x",
        type = "Positive"
    )
    masterObjVal = Variable(
        m,
        name = "masterObjVal"
    )
    alpha = Variable(
        m,
        name="alpha",
    )

    # Parameter
    dualVar = Parameter(
        m,
        name = "dualVar",
        domain = iter,
        description = "dual variables of constraint 5"
    )
    subProbHistoric = Parameter(
        m,
        name = "subProbHistoric",
        domain = iter,
        description = "y values that results in optimal value of subproblem"
    )
    allOptimalX = Parameter(
        m,
        name = "allOptimalX",
        domain = iter,
        description = "x values that results in optimal value of subproblem"
    )
    
    # Equation
    masterobj = Equation(
        m,
        name = "masterobj",
        description = "master objective function"
    )
    cutConstraints = Equation(
        m,
        name = "cutConstraints",
        description = "Bender cuts",
        domain = iter
    )
    
    masterobj[...] = masterObjVal == -(1/4)*x + alpha
    cutConstraints[activeIter] = alpha >=  subProbHistoric[activeIter
    ] + dualVar[activeIter] * (x - allOptimalX[activeIter])
    print("Active iter should go here")
    print(activeIter.records)
    alpha.lo = -25
    x.up = 16
    
    masterproblem = Model(
        m,
        name = "masterproblem",
        equations = [cutConstraints, masterobj],
        problem = "LP",
        sense = Sense.MIN,
        objective = masterObjVal
    )
    ####################### Subproblem #######################
    # Variables
    subObjVal = Variable(
        m,
        name = "subObjVal"
    )
    x2 = Variable(
        m, 
        name = "x2",
        type = "Positive"
    )
    y = Variable(
        m, 
        name = "y",
        type = "Positive"
    )
    
    # Equations
    subObj = Equation(
        m,
        name = "subObj",
        description = "subproblem objective function"
    )
    con1 = Equation(
        m,
        name="con1"
    )
    con2 = Equation(
        m,
        name="con2"
    )
    con3 = Equation(
        m,
        name="con3"
    )
    con4 = Equation(
        m,
        name="con4"
    )
    con5 = Equation(
        m,
        name="con5"
    )
    
    # Equations
    subObj[...] = subObjVal == -y
    con1[...] = y - x2 <= 5
    con2[...] = y -(1/2) * x2 <= (15/2)
    con3[...] = y + (1/2) * x2 <= (35/2)
    con4[...] = -y + x2 <= 10
    con5[...] = x2 == x.l
    
    subproblem = Model(
        m,
        name = "subproblem",
        equations = [subObj, con1, con2, con3, con4, con5],
        problem = "LP",
        sense = Sense.MIN,
        objective = subObjVal
    )
    
    # Scalars for bookeeping
    rgap = Parameter(m, name="rgap", records=0, description="relative gap")
    lowerBound = Parameter(
        m,
        name="lowerBound",
        records=float("-inf"),
        description="global lower bound",
    )
    upperBound = Parameter(
        m,
        name="upperBound",
        records=float("inf"),
        description="global upper bound",
    )
    objMaster = Parameter(m, name="objMaster", records=0)
    objSub = Parameter(m, name="objSub", records=0)
    rtol = 0.001

    # Solve Initial master problem
    masterproblem.solve()
    print(masterproblem.objective_value)
    objMaster.setRecords(masterObjVal.records["level"])
    lowerBound.setRecords(masterObjVal.records["level"])
    allOptimalX["1"] = x.records["level"][0]
    for iteration, _ in iter.records.itertuples(index = False):
        print("\n\n ....................................................")
        print("Iteration: " + iteration)
        subproblem.solve()
        objSub[...] = subObjVal.l
        dualVar[iteration] = con5.m
        subProbHistoric[iteration] = subObjVal.l
        print(objMaster.records)
        print(alpha.records.level[0])
        print(subObjVal.records.level[0])
        print("Value of x2 is: ", x2.records.level[0])
        print("Value of y is: ", y.records.level[0])
        upperBound[...] = objMaster - alpha.l + subObjVal.l
        activeIter[iteration] = True
        print(activeIter.records)
        print("Upper bound is ")
        print(upperBound.records.values[0][0])
        print("Lower bound is " )
        print(lowerBound.records.values[0][0])
        
        if (upperBound.records.values[0][0] - lowerBound.records.values[0][0] < rtol):
            print(masterproblem.objective_value)
            print("Rtol is " + str(rtol))
            break
        masterproblem.solve()
        print("\nAfter solving masterproblem, objval is : ", masterObjVal.records["level"][0])
        print("Alpha is : ", alpha.records["level"][0])
        print("x is: ", x.records["level"][0])
        print("SubprobHistoric: ", subProbHistoric.records)
        print("allOptimalX[activeIter]: ", allOptimalX.records)
        #cutConstraints[activeIter] = alpha >=  subProbHistoric[activeIter
    #] + dualVar[activeIter] * (x - allOptimalX[activeIter])
        boo = vars(cutConstraints)
        print("DualVar should be 0.5 but is: ", dualVar.records.values)
    
        # On the first iteration, this master proble messed up lower bound
        lowerBound.setRecords(masterObjVal.records["level"])
        objMaster.setRecords(masterObjVal.records["level"])
        allOptimalX[str(int(iteration) + 1)] = x.records["level"][0]
        
if __name__ == "__main__":
    main()

/Users/hongkai/miniconda3/envs/optimization/lib/python3.11/site-packages/gamspy/_container.py:117: UserWarning: Delayed execution mode will be deprecated in 0.12.0.
  warnings.warn(


Active iter should go here
None
-29.0


 ....................................................
Iteration: 1
   value
0  -29.0
-25.0
-9.5
Value of x2 is:  16.0
Value of y is:  9.5
  iter element_text
0    1             
Upper bound is 
-13.5
Lower bound is 
-29.0

After solving masterproblem, objval is :  -17.5
Alpha is :  -17.5
x is:  0.0
SubprobHistoric:    iter  value
0    1   -9.5
allOptimalX[activeIter]:    iter  value
0    1   16.0
DualVar should be 0.5 but is:  [['1' 0.5]]


 ....................................................
Iteration: 2
   value
0  -17.5
-17.5
-5.0
Value of x2 is:  0.0
Value of y is:  5.0
  iter element_text
0    1             
1    2             
Upper bound is 
-5.0
Lower bound is 
-17.5

After solving masterproblem, objval is :  -15.416666666666666
Alpha is :  -13.333333333333332
x is:  8.333333333333332
SubprobHistoric:    iter  value
0    1   -9.5
1    2   -5.0
allOptimalX[activeIter]:    iter  value
0    1   16.0
1    2   -0.0
DualVar should be 0.5 but i

# Example from GAMSPY

In [88]:
"""
## GAMSSOURCE: https://www.gams.com/latest/gamslib_ml/libhtml/gamslib_spbenders1.html
## LICENSETYPE: Demo
## MODELTYPE: LP
## KEYWORDS: linear programming, stochastic Benders algorithm, transportation problem


Stochastic Benders - Sequential GAMS Loop (SPBENDERS1)

This example demonstrates a stochastic Benders implementation for the
simple transport example.

This is the first example of a sequence of stochastic Benders
implementations using various methods to solve the master and
subproblem.

This first example implements the stochastic Benders algorithm using
sequential solves of the master and subproblems in a GAMS loop.
"""

from __future__ import annotations

import os

import pandas as pd

import gamspy.math as gams_math
from gamspy import Container
from gamspy import Equation
from gamspy import Model
from gamspy import Parameter
from gamspy import Sense
from gamspy import Set
from gamspy import Sum
from gamspy import Variable


def main():
    m = Container(
        system_directory=os.getenv("SYSTEM_DIRECTORY", None),
        delayed_execution=int(os.getenv("DELAYED_EXECUTION", False)),
    )

    # Prepare data
    cost = pd.DataFrame([
        ["f1", "d1", 2.49],
        ["f1", "d2", 5.21],
        ["f1", "d3", 3.76],
        ["f1", "d4", 4.85],
        ["f1", "d5", 2.07],
        ["f2", "d1", 1.46],
        ["f2", "d2", 2.54],
        ["f2", "d3", 1.83],
        ["f2", "d4", 1.86],
        ["f2", "d5", 4.76],
        ["f3", "d1", 3.26],
        ["f3", "d2", 3.08],
        ["f3", "d3", 2.60],
        ["f3", "d4", 3.76],
        ["f3", "d5", 4.45],
    ])

    scenarios = pd.DataFrame([
        ["lo", "d1", 150],
        ["lo", "d2", 100],
        ["lo", "d3", 250],
        ["lo", "d4", 300],
        ["lo", "d5", 600],
        ["lo", "prob", 0.25],
        ["mid", "d1", 160],
        ["mid", "d2", 120],
        ["mid", "d3", 270],
        ["mid", "d4", 325],
        ["mid", "d5", 700],
        ["mid", "prob", 0.50],
        ["hi", "d1", 170],
        ["hi", "d2", 135],
        ["hi", "d3", 300],
        ["hi", "d4", 350],
        ["hi", "d5", 800],
        ["hi", "prob", 0.25],
    ])

    cut_coefficients = pd.DataFrame([
        [idx, f"d{center}", 0]
        for idx in range(1, 26)
        for center in range(1, 6)
    ])

    # Set
    i = Set(m, name="i", records=["f1", "f2", "f3"], description="factories")
    j = Set(
        m,
        name="j",
        records=["d1", "d2", "d3", "d4", "d5"],
        description="distribution centers",
    )
    s = Set(m, name="s", records=["lo", "mid", "hi"], description="scenarios")

    # Data
    capacity = Parameter(
        m,
        name="capacity",
        domain=i,
        records=pd.DataFrame([["f1", 500], ["f2", 450], ["f3", 650]]),
        description="unit capacity at factories",
    )
    demand = Parameter(
        m,
        name="demand",
        domain=j,
        records=pd.DataFrame(
            [["d1", 160], ["d2", 120], ["d3", 270], ["d4", 325], ["d5", 700]]
        ),
        description="unit demand at distribution centers",
    )
    prodcost = Parameter(
        m, name="prodcost", records=14, description="unit production cost"
    )
    price = Parameter(m, name="price", records=24, description="sales price")
    wastecost = Parameter(
        m,
        name="wastecost",
        records=4,
        description="cost of removal of overstocked products",
    )
    transcost = Parameter(
        m,
        name="transcost",
        domain=[i, j],
        records=cost,
        description="unit transportation cost",
    )
    ScenarioData = Parameter(
        m,
        name="scenariodata",
        domain=[s, "*"],
        records=scenarios,
        description="possible outcomes for demand plus probabilities",
    )

    # Set
    iter = Set(
        m,
        name="iter",
        records=[f"{idx}" for idx in range(1, 26)],
        description="max Benders iterations",
    )
    itActive = Set(
        m, name="itActive", domain=iter, description="active Benders cuts"
    )

    # Parameter
    cutconst = Parameter(
        m,
        name="cutconst",
        domain=iter,
        records=pd.DataFrame([[f"{idx}", 0] for idx in range(1, 26)]),
        description="constants in optimality cuts",
    )
    cutcoeff = Parameter(
        m,
        name="cutcoeff",
        domain=[iter, j],
        records=cut_coefficients,
        description="coefficients in optimality cuts",
    )

    # Variable
    ship = Variable(
        m, name="ship", domain=[i, j], type="Positive", description="shipments"
    )
    product = Variable(m, name="product", domain=i, description="production")
    received = Variable(
        m, name="received", domain=j, description="quantity sent to market"
    )
    zmaster = Variable(
        m, name="zmaster", description="objective variable of master problem"
    )
    theta = Variable(m, name="theta", description="future profit")

    # Equation
    masterobj = Equation(
        m, name="masterobj", description="master objective function"
    )
    production = Equation(
        m,
        name="production",
        domain=i,
        description="calculate production in each factory",
    )
    receive = Equation(
        m,
        name="receive",
        domain=j,
        description="calculate quantity to be send to markets",
    )
    optcut = Equation(
        m, name="optcut", domain=iter, description="Benders optimality cuts"
    )

    masterobj[...] = zmaster == theta - Sum(
        (i, j), transcost[i, j] * ship[i, j]
    ) - Sum(i, prodcost * product[i])
    receive[j] = received[j] == Sum(i, ship[i, j])
    production[i] = product[i] == Sum(j, ship[i, j])
    optcut[itActive] = theta <= cutconst[itActive] + Sum(
        j, cutcoeff[itActive, j] * received[j]
    )
    product.up[i] = capacity[i]

    masterproblem = Model(
        m,
        name="masterproblem",
        equations=m.getEquations(),
        problem="LP",
        sense=Sense.MAX,
        objective=zmaster,
    )

    # Variable
    sales = Variable(
        m,
        name="sales",
        domain=j,
        type="Positive",
        description="sales (actually sold)",
    )
    waste = Variable(
        m,
        name="waste",
        domain=j,
        type="Positive",
        description="overstocked products",
    )
    zsub = Variable(
        m, name="zsub", description="objective variable of sub problem"
    )

    # Equation
    subobj = Equation(
        m, name="subobj", description="subproblem objective function"
    )
    selling = Equation(
        m, name="selling", domain=j, description="part of received is sold"
    )
    market = Equation(
        m, name="market", domain=j, description="upperbound on sales"
    )

    subobj[...] = zsub == Sum(j, price * sales[j]) - Sum(
        j, wastecost * waste[j]
    )
    selling[j] = sales[j] + waste[j] == received.l[j]
    market[j] = sales[j] <= demand[j]

    subproblem = Model(
        m,
        name="subproblem",
        equations=[subobj, selling, market],
        problem="LP",
        sense=Sense.MAX,
        objective=zsub,
    )

    # Scalar
    rgap = Parameter(m, name="rgap", records=0, description="relative gap")
    lowerBound = Parameter(
        m,
        name="lowerBound",
        records=float("-inf"),
        description="global lower bound",
    )
    upperBound = Parameter(
        m,
        name="upperBound",
        records=float("inf"),
        description="global upper bound",
    )
    objMaster = Parameter(m, name="objMaster", records=0)
    objSub = Parameter(m, name="objSub", records=0)
    rtol = 0.001

    received.l[j] = 0

    for iteration, _ in iter.records.itertuples(index=False):
        objSub[...] = 0

        for scenario, _ in s.records.itertuples(index=False):
            demand[j] = ScenarioData[scenario, j]
            subproblem.solve()
            objSub[...] = objSub + ScenarioData[scenario, "prob"] * zsub.l
            cutconst[iteration] = cutconst[iteration] + ScenarioData[
                scenario, "prob"
            ] * Sum(j, market.m[j] * demand[j])
            cutcoeff[iteration, j] = (
                cutcoeff[iteration, j]
                + ScenarioData[scenario, "prob"] * selling.m[j]
            )

        itActive[iteration] = True

        if (
            lowerBound.records.values[0][0]
            < objMaster.records.values[0][0] + objSub.records.values[0][0]
        ):
            lowerBound[...] = objMaster + objSub

        rgap[...] = (upperBound - lowerBound) / (1 + gams_math.abs(upperBound))
        if rgap.records.values[0][0] < rtol:
            print("Lower Bound is: " )
            print(lowerBound.records.values)
            break
        
        masterproblem.solve()
        upperBound.setRecords(zmaster.records["level"])
        objMaster.setRecords(zmaster.records["level"] - theta.records["level"])


if __name__ == "__main__":
    main()

Replace this: 
$$\underset{x}{\text{minimize}} \;\;c^T + E_\omega(Q(x, \omega))$$
With this: 
$$\underset{x}{\text{minimize}} \;\;c^T + CVaR_\alpha(Q(x, \omega))$$
